In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [48]:
df = pd.read_csv('N:/Springboard/Second_Capstone/raw_data/Post_EDA_Data.csv')
df.describe()

,Unnamed: 0,Unnamed: 0.1,yr,mo,dy,state_fips,state_num,mag,inj,fat,...,county_11,state_1,state_2,state_3,track_pop,track_area,track_pop_den,decade,USD_prop_dam,hour
count,35877.00000,35877.000000,35877.000000,35877.000000,35877.000000,35877.000000,35877.000000,35877.000000,35877.000000,35877.000000,...,0.0,35877.000000,310.000000,6.000000,3.587700e+04,35877.000000,35767.000000,35877.000000,3.587700e+04,35877.000000
mean,17938.00000,32345.672130,1990.093124,5.971820,15.862558,24.895476,20.402458,0.765393,1.395964,0.089444,...,NaN,24.878474,26.193548,26.333333,1.801314e+05,1253.868640,196.821718,1985.523873,1.291128e+07,15.072024
std,10356.94214,18733.366346,18.893374,2.435009,8.809200,14.115753,24.361608,0.892704,17.104788,1.297462,...,NaN,14.106606,15.772286,18.315749,5.184175e+05,1375.381516,639.595502,18.773490,6.052032e+07,4.997373
min,0.00000,1.000000,1950.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,1.000000,1.000000,4.000000,0.000000e+00,0.000000,0.200000,1950.000000,0.000000e+00,0.000000
25%,8969.00000,16350.000000,1975.000000,4.000000,8.000000,12.000000,2.000000,0.000000,0.000000,0.000000,...,NaN,12.000000,13.000000,13.500000,1.119872e+04,620.290000,12.900000,1970.000000,0.000000e+00,13.000000
50%,17938.00000,32249.000000,1993.000000,6.000000,16.000000,22.000000,12.000000,1.000000,0.000000,0.000000,...,NaN,22.000000,29.000000,23.500000,3.656883e+04,876.820000,45.700000,1990.000000,5.000000e+03,16.000000
75%,26907.00000,48573.000000,2006.000000,7.000000,24.000000,39.000000,30.000000,1.000000,0.000000,0.000000,...,NaN,39.000000,40.000000,42.500000,1.325231e+05,1362.040000,143.100000,2000.000000,5.000000e+06,18.000000
max,35876.00000,65161.000000,2019.000000,12.000000,31.000000,56.000000,232.000000,5.000000,1500.000000,94.000000,...,NaN,56.000000,54.000000,48.000000,1.115306e+07,41667.940000,35369.200000,2010.000000,5.000000e+09,23.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35877 entries, 0 to 35876
Data columns (total 46 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       35877 non-null  int64  
 1   Unnamed: 0.1     35877 non-null  int64  
 2   temp_id          35877 non-null  object 
 3   yr               35877 non-null  int64  
 4   mo               35877 non-null  int64  
 5   dy               35877 non-null  int64  
 6   date             35877 non-null  object 
 7   time             35877 non-null  object 
 8   st               35877 non-null  object 
 9   state_fips       35877 non-null  int64  
 10  state_num        35877 non-null  int64  
 11  mag              35877 non-null  int64  
 12  inj              35877 non-null  int64  
 13  fat              35877 non-null  int64  
 14  loss             35877 non-null  float64
 15  start_lat        35877 non-null  float64
 16  start_lon        35877 non-null  float64
 17  end_lat     

In [50]:
df['mag'].value_counts()

0    17070
1    12192
2     4931
3     1359
4      296
5       29
Name: mag, dtype: int64

Examining the magnitude column shows us that approximately 17K of our total ~36K rows contain a magnitude value of 0. We know that the EF-scale runs from 1-5, so these values are technically incorrect, and we don't want to use them for prediction purposes.

In the interest of not losing relevant data, however, we would like to isolate instances of 0 magnitude that still have data in our target variables (inj and fat). We will change those records to the next most common magnitude (1, in this case), and then drop the rest.

In [51]:
df_zmag = df.loc[df['mag'] == 0]
print(df_zmag['inj'].sum())
print(df_zmag['fat'].sum())
print(df_zmag['USD_prop_dam'].sum())

472
16
156867342600


In [52]:
df.loc[(df['mag']==0) & ((df['inj']!=0) | (df['fat']!=0) | (df['USD_prop_dam']!=0)),'mag'] = 1
df['mag'].value_counts()

1    18578
0    10684
2     4931
3     1359
4      296
5       29
Name: mag, dtype: int64

In [53]:
df = df[df.mag != 0]
df['mag'].value_counts()

1    18578
2     4931
3     1359
4      296
5       29
Name: mag, dtype: int64

One additional feature that might be beneficial for prediction is the number of counties a tornado enters. This could be potentially operate as a bridge between track length and population density, as smaller counties tend to be more populous.

In [54]:
df_counties = df[['county_1','county_2', 'county_3', 'county_4', 'county_5', 'county_6', 'county_7', 'county_8', 'county_9', 'county_10', 'county_11']]
df['no_counties'] = df_counties.count(axis=1)
df.head()

,Unnamed: 0,Unnamed: 0.1,temp_id,yr,mo,dy,date,time,st,state_fips,...,track_area,track_pop_den,wet_or_dry_yr,precip_class,warm_or_cool_yr,temp_class,decade,USD_prop_dam,hour,no_counties
0,0,1,1951_1,1951,1,6,1/6/1951,17:30:00,LA,22,...,2026.64,99.9,Normal,Wet,Normal,Normal,1950,500000,17,2
1,1,2,1952_1,1952,1,17,1/17/1952,19:34:00,PA,42,...,1037.82,87.7,Normal,Normal,Normal,Normal,1950,500000,19,1
2,2,3,1953_1,1953,1,7,1/7/1953,20:15:00,AR,5,...,598.84,14.5,Normal,Normal,Normal,Warm,1950,5000,20,1
4,4,6,1956_1,1956,1,28,1/28/1956,21:30:00,AR,5,...,1706.48,174.8,Normal,Wet,Normal,Warm,1950,50000,21,2
5,5,7,1957_1,1957,1,21,1/21/1957,13:20:00,HI,15,...,2126.98,1586.7,Wet,Wet,Normal,Normal,1950,50000,13,1


Our dataframe contains several intermediate columns that were used in the creation of others, but are not suitable to be included as features in our modeling. We will remove those now, leaving the following features intact:

1. yr
2. mo
3. mag
4. len
5. wid
6. number of counties
7. track_pop
8. track_area
9. track_pop_den
10. wet_or_dry_yr
11. precip_class
12. warm_or_cool_yr
13. temp_class
14. USD_prop_dam
15. hour
16. start_lat
17. start_lon

We also need to identify our target variable, which is this case will be a combination of injuries and fatalities.

In [55]:
df_ml = df[['inj', 'fat', 'yr', 'mo', 'mag', 'len', 'wid', 'no_counties', 'track_pop', 'track_area', 'track_pop_den', 'wet_or_dry_yr', 'precip_class', 'warm_or_cool_yr', 'temp_class', 'USD_prop_dam', 'hour', 'start_lat', 'start_lon']]
df_ml.head()

,inj,fat,yr,mo,mag,len,wid,no_counties,track_pop,track_area,track_pop_den,wet_or_dry_yr,precip_class,warm_or_cool_yr,temp_class,USD_prop_dam,hour,start_lat,start_lon
0,11,0,1951,1,3,16.8,317,2,159129.416,2026.64,99.9,Normal,Wet,Normal,Normal,500000,17,31.30,-92.47
1,0,0,1952,1,2,0.5,400,1,91016.814,1037.82,87.7,Normal,Normal,Normal,Normal,500000,19,41.60,-80.30
2,2,0,1953,1,2,4.5,100,1,8683.180,598.84,14.5,Normal,Normal,Normal,Warm,5000,20,33.83,-92.17
4,12,0,1956,1,3,23.0,150,2,193736.975,1706.48,174.8,Normal,Wet,Normal,Warm,50000,21,35.20,-92.45
5,0,0,1957,1,1,2.5,10,1,3374879.166,2126.98,1586.7,Wet,Wet,Normal,Normal,50000,13,21.52,-158.07


The last issue we need to deal with prior to beginning our preprocessing is that we technically have 2 target variables in Injuries and Fatalities when we need only 1 for modeling. It doesn't make sense to remove either column from consideration, so instead we'll make a new target variable for human toll (hum_toll) that is a weighted blend of injuries and fatalities. We'll proceed with a full 1 for fatalities, and 0.1 for injuries.

In [56]:
df_ml['hum_toll'] = df['fat'] + 0.1*df['inj']
df_ml = df_ml.drop(df_ml[['fat']], 1)
df_ml = df_ml.drop(df_ml[['inj']], 1)
df_ml.head()

C:\Users\Mavis\AppData\Local\Temp/ipykernel_756/24882677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml['hum_toll'] = df['fat'] + 0.1*df['inj']
C:\Users\Mavis\AppData\Local\Temp/ipykernel_756/24882677.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_ml = df_ml.drop(df_ml[['fat']], 1)
C:\Users\Mavis\AppData\Local\Temp/ipykernel_756/24882677.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_ml = df_ml.drop(df_ml[['inj']], 1)


,yr,mo,mag,len,wid,no_counties,track_pop,track_area,track_pop_den,wet_or_dry_yr,precip_class,warm_or_cool_yr,temp_class,USD_prop_dam,hour,start_lat,start_lon,hum_toll
0,1951,1,3,16.8,317,2,159129.416,2026.64,99.9,Normal,Wet,Normal,Normal,500000,17,31.30,-92.47,1.1
1,1952,1,2,0.5,400,1,91016.814,1037.82,87.7,Normal,Normal,Normal,Normal,500000,19,41.60,-80.30,0.0
2,1953,1,2,4.5,100,1,8683.180,598.84,14.5,Normal,Normal,Normal,Warm,5000,20,33.83,-92.17,0.2
4,1956,1,3,23.0,150,2,193736.975,1706.48,174.8,Normal,Wet,Normal,Warm,50000,21,35.20,-92.45,1.2
5,1957,1,1,2.5,10,1,3374879.166,2126.98,1586.7,Wet,Wet,Normal,Normal,50000,13,21.52,-158.07,0.0


In [57]:
df_ml = pd.get_dummies(data=df_ml, columns=['wet_or_dry_yr', 'precip_class', 'warm_or_cool_yr', 'temp_class'])

In [58]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25193 entries, 0 to 35876
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   yr                      25193 non-null  int64  
 1   mo                      25193 non-null  int64  
 2   mag                     25193 non-null  int64  
 3   len                     25193 non-null  float64
 4   wid                     25193 non-null  int64  
 5   no_counties             25193 non-null  int64  
 6   track_pop               25193 non-null  float64
 7   track_area              25193 non-null  float64
 8   track_pop_den           25135 non-null  float64
 9   USD_prop_dam            25193 non-null  int64  
 10  hour                    25193 non-null  int64  
 11  start_lat               25193 non-null  float64
 12  start_lon               25193 non-null  float64
 13  hum_toll                25193 non-null  float64
 14  wet_or_dry_yr_Dry       25193 non-null

Now that the data is in the format we want, it's time to begin preprocessing. The process:
1. Split our dataframe into features and target variable (X and y)
2. Initialize a StandardScaler object
3. Split the data into Train and Test datasets. We'll use a 75/25 train/test split.
4. Fit the scaler to the training data and transform it, and then use that same scaler to transform the test data.

In [59]:
X = df_ml.drop('hum_toll', axis=1).values
y = df_ml['hum_toll']

In [60]:
scaler = StandardScaler()

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [62]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform (X_test)